# Aggregation and Grouping

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
planets = sns.load_dataset('planets')

**Simple Aggregation in Pandas**  
For a DataFrame, by default the aggregates return results within each column.

In [ ]:
df.mean(axis='columns')
planets.dropna().describe()

![](2.jpg)

**GroupBy: Split, Apply, Combine**

In [ ]:
planets.groupby('method').mean()
planets.groupby('method')['orbital_period'].median()

The GroupBy object supports direct iteration over the groups, returning each group as a Series or DataFrame:

In [ ]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape{1}".format(method, group.shape))

**Aggregate, filter, transform, apply**

In [11]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


**1. Aggregate**  
Aggregate() can take a string, a function, or a list thereof, and compute all the aggregates at once. Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})

**2. Filtering**  
A filtering operation allows you to drop data based on the group properties.

In [14]:
def filter_func(x):
    return x['data2'].std() > 4
df.groupby('key').filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


**3. Transformation**  
While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.

**The difference between transforma() and apply()!!!**
https://blog.csdn.net/qq_40587575/article/details/81204514

In [22]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


**4. The apply() method**  
The **apply()** method lets you apply an arbitrary function to the group results. The function should take a DataFrame, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned.

In [ ]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x
df.groupby('key').apply(norm_by_data2)

**Specify the split key**

**1. A list, series, or index providing the grouping keys**

In [ ]:
L = [0, 1, 0, 1, 2, 0]
df.groupby(L).sum()

**2. A dictionary or series mapping index to group**

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
df2.groupby(mapping).sum()

**3. Any Python function**

In [ ]:
df2.groupby(str.lower).mean()

**4. A list of valid keys**  
Any of the preceding key choices can be combined to group on a multi-index.

In [ ]:
df2.groupby([str.lower, mapping]).mean()

**Grouping Example**

In [49]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


In [3]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [4]:
df.groupby('key').aggregate('describe')

data1                                          data2                      \
    count mean      std  min   25%  50%   75%  max count mean       std  min   
key                                                                            
A     2.0  1.5  2.12132  0.0  0.75  1.5  2.25  3.0   2.0  4.0  1.414214  3.0   
B     2.0  2.5  2.12132  1.0  1.75  2.5  3.25  4.0   2.0  3.5  4.949747  0.0   
C     2.0  3.5  2.12132  2.0  2.75  3.5  4.25  5.0   2.0  6.0  4.242641  3.0   

                           
      25%  50%   75%  max  
key                        
A    3.50  4.0  4.50  5.0  
B    1.75  3.5  5.25  7.0  
C    4.50  6.0  7.50  9.0